In [1]:
!python -m pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-6y79pm_e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-6y79pm_e
  Resolved https://github.com/huggingface/transformers to commit ca03842cdcf2823301171ab27aec4b6b1cafdbc1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
pip install sentencepiece

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install 'accelerate>=0.26.0'

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from PIL import Image
import time
import json
import os

# Step 1: Load the processor and model
print("Loading processor and model...")

# Load the processor and model from pre-trained weights
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

# Load the model with float16 precision and transfer it to GPU
model = LlavaNextForConditionalGeneration.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf", 
    torch_dtype=torch.float16,  # Using float16 precision for GPU
    low_cpu_mem_usage=True
)
model.to("cuda:0")  # Move model to GPU

print("Processor and model loaded successfully.")

# Step 2: Load dataset from JSON
json_path = "dataset_fixeddd.json"  # Path to your JSON file
with open(json_path, "r") as f:
    dataset = json.load(f)

results = []

# Step 3: Process each image and its questions
for item in dataset:
    image_path = item['image']
    questions = [q["question"] for q in item['questions']]
    groundtruth_answers = [q["answer"] for q in item['questions']]

    # Check if the image file exists
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        continue

    print(f"Processing image {image_path}...")
    image = Image.open(image_path)

    answers = []
    start_time = time.time()

    for question in questions:
        # Format the conversation for each question
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question},
                    {"type": "image"}
                ]
            }
        ]
        prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

        # Prepare inputs for the model
        inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda:0")

        # Generate response
        output = model.generate(**inputs, max_new_tokens=600)
        decoded_output = processor.decode(output[0], skip_special_tokens=True)

        answers.append(decoded_output)  # Collect the answer

    end_time = time.time()

    # Append results
    results.append({
        "image": image_path,
        "questions": questions,
        "generated_answers": answers,
        "groundtruth_answers": groundtruth_answers
    })

# Step 4: Save results to a new JSON file
output_json_path = "llava_generated_answers.json"
with open(output_json_path, "w") as f:
    json.dump(results, f, indent=4)

print(f"Processing complete. Results saved to {output_json_path}.")


Loading processor and model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processor and model loaded successfully.
Processing image datasett/datasetUML1.png...


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
Expanding inputs for image tokens in LLaVa-NeXT should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.50.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for 

...

Processing image datasett/datasetUML19.png...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing image datasett/datasetUML20.png...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing complete. Results saved to llava_generated_answers.json.
